# 기계학습과 응용 기말 프로젝트

## 2018131027 손홍구

## Riot API를 활용한 사용자 승률 예측 및 개선 방안 토의

### 서론
이번 프로젝트의 주된 목적은 머신러닝을 통한 롤이라는 게임의 승률 예측이며,  
최상위 플레이어의 승률 공식을 이용하여 머신러닝을 진행할 것이다.  
하위 티어의 플레이어가 게임을 진행하면서 자신의 게임 기록을 입력했을 때,  
정확도가 높을 수록(상위 티어와의 플레이가 유사할 수록) 실력이 올라갔음을 나타내는 지표가 될 것이다.  

이를 위해 롤을 서비스하는 기업에서 제공하는 무료 api인 Riot api를 활용할 것이다.  
  
대략적인 설명은 서론에서 진행하겠지만, 더욱 자세한 설명을 위해 코드마다 MarkDown을 통해 설명할 것이다.  
  
Riot api를 활용하는 방법은 다음과 같다.  
1- 모든 롤 플레이어는 자신의 고유 닉네임을 갖고있다.  
2- 하지만 서비스하는 대륙이 다를 경우(ex. 아시아, 미국 등) 고유 닉네임은 중복될 수 있다.  
3- 따라서 Riot에서는 유저의 게임 기록을 닉네임으로 저장하지 않는다.  
4- Riot은 'puuid'라는 고유 id를 사용하며, 이는 전세계 모든 플레이어마다 고유로 갖고있는 값이다.  
5- 각 플레이어의 게임 플레이 기록을 검색하기 위해서는 이 puuid를 활용해야만한다.  
6- 따라서 (1)에서는 puuid를 얻어오는 방법에 대해 먼저 소개한다.  
7- 이후, 가장 게임이 시스템적으로 정형화된 최상위 플레이어(한국 서버 기준 300등 이내)의 모든  
   게임 기록을 가져온다.  
8- 7에 대한 이유는, 가장 실력적으로 최상위에 있기 때문에, 게임 내에서 '승리'하기 위한 방법만을 사용한다.  
9- 하위 티어 플레이어의 경우, 승리보다는 자신의 재미를 추구하는 경우가 많기 때문에 승리에 따른 지표에  
   대한 신뢰도가 낮다.  
10- 따라서, 최상위 플레이어들의 게임 지표만을 이용하여 데이터를 정제한다.  
11- 이후 sklearn을 통해 머신러닝을 진행하고, 신뢰할만한 훈련이 되었는지 테스트한다.  
12- 마지막으로 본인의 아이디를 직접 입력하여, 게임 데이터를 넣었을 때 정확도를 확인한다.  
13- 이 정확도가 높으면 높을 수록, 챌린저(최상위권)에 조금 더 가까운 실력을 가졌다고 결론낼 수 있다.
 

### 주의

Riot api는 개인 사용자들에게도 무료로 배포되는 api이지만, 일정 시간이 지나면 다시 api에 접근하기 위한  
key를 재발급받아야하고, 이는 라이엇 홈페이지에 직접 로그인해야만 가능합니다.  
이에 (1)~(2)의 데이터를 검색하고 정제하는 과정을 재실행하기 위해서는 api접근 key를 재발급받아야합니다.  
접근key는 https://developer.riotgames.com/ 에서 발급받을 수 있습니다.
  
또한 매 iteration마다, time sleep(5)를 설정해두었는데,  
Riot에서 매 request마다 딜레이를 가진 이후 응답하기 때문입니다.  
(약 100개의 요청 당 2분이라고 알려져있어, 오류를 없애기 위해 넉넉하게 설정했습니다.) 

### 1. 플레이어 닉네임으로 정보 검색하기

In [ ]:
import pandas as pd
import requests
import json
from pandas import json_normalize

In [ ]:
searchid = input('검색하고 싶은 소환사명을 입력해주세요')

검색하고 싶은 소환사명을 입력해주세요호걸쾌남


In [ ]:
api_key = 'RGAPI-5ae606a0-3159-4daf-88a4-8e6ef6226c39'
sohwan = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" +searchid +'?api_key=' + api_key
r = requests.get(sohwan)
player = json.loads(r.content)
player
#아래와 같은 유저에 대한 정보들을 얻을 수 있다.
#하지만 우리의 목표는 승률 예측이기 때문에, 플레이 기록 검색에 필수적인 puuid를 얻는 방법만 살펴본다.

{'id': 'ldUVYr6rlcZ2MKDja9snxX4gnVYdWVGX2yo-5PvcIqfQQR8',
 'accountId': 'GDiXNYMDg4PV3Cxu4XIGl97n61v9rwVlSEcA2bVu4qHK',
 'puuid': 'vV1pJUL7R2TJA2qm-dIdum7-XcM6LWVGD8SzJ5lFukEXd_JyfiCadk8mqxP9q2uRZ7068weLeLaJsQ',
 'name': '호걸쾌남',
 'profileIconId': 5026,
 'revisionDate': 1670169369000,
 'summonerLevel': 393}

In [ ]:
playerid = json.loads(r.content)['puuid']
playerid
#아래와 같은 고유아이디가 puuid이다.

'vV1pJUL7R2TJA2qm-dIdum7-XcM6LWVGD8SzJ5lFukEXd_JyfiCadk8mqxP9q2uRZ7068weLeLaJsQ'

### 2. 챌린저 플레이 데이터 다운받기

In [ ]:
#이제 puuid를 얻는 방법을 살펴봤으니, 상위 300등에 위치한 플레이어들(챌린저)의 아이디를 얻어야한다.
#이 때 Riot은 최상위권 리그에 대한 별개의 api를 제공한다.
#300명의 플레이어에 대한 정보를 제공하는 챌린저api를 이용한다.
challenger = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(challenger)
league_df = pd.DataFrame(r.json())
league_df.reset_index(inplace=True)
league_df
#아래와 같은 300명의 데이터를 얻게 되었다.

,index,tier,leagueId,queue,name,entries
0,0,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'KEKtyDMgTJUcPhfsHztLtpU98KyIeC...
1,1,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': '97XkEPTUfhI8BK192Bakb9Xeqhd9BX...
2,2,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': '3OHO1cpkydLO2EdSKmigx_vy79Tg_k...
3,3,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'EXazcoG7DZN9OJL05TN0cXFv9f0QoM...
4,4,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'jxQK0sGhLCW23SaTJ7m4d2EV23-Rkr...
...,...,...,...,...,...,...
295,295,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'qHT9-KIZ5QG4gvuD20fjMwqDLV2Goz...
296,296,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'COgQT7YJapv-WkU7gUg8t8MGYdWxh5...
297,297,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'DjeJeRD0dipXpP7yHeWtshisALDict...
298,298,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'sBjz6K6MuPtOo2P7DMzhwMFJtQ5xhS...


In [ ]:
#하지만, 우리가 찾고있는 정보가 가시적으로 보이지 않는다.
#원하는 정보는 entries column에 딕셔너리 형태로 저장되어 있다.
league_df['entries']

0      {'summonerId': 'KEKtyDMgTJUcPhfsHztLtpU98KyIeC...
1      {'summonerId': '97XkEPTUfhI8BK192Bakb9Xeqhd9BX...
2      {'summonerId': '3OHO1cpkydLO2EdSKmigx_vy79Tg_k...
3      {'summonerId': 'EXazcoG7DZN9OJL05TN0cXFv9f0QoM...
4      {'summonerId': 'jxQK0sGhLCW23SaTJ7m4d2EV23-Rkr...
                             ...                        
295    {'summonerId': 'qHT9-KIZ5QG4gvuD20fjMwqDLV2Goz...
296    {'summonerId': 'COgQT7YJapv-WkU7gUg8t8MGYdWxh5...
297    {'summonerId': 'DjeJeRD0dipXpP7yHeWtshisALDict...
298    {'summonerId': 'sBjz6K6MuPtOo2P7DMzhwMFJtQ5xhS...
299    {'summonerId': 'VAPWbaRD6jLk4DMQD2NHCvoec9KTUD...
Name: entries, Length: 300, dtype: object

In [ ]:
#이를 이용하기 위해 딕셔너리를 풀어서 다시 기존 데이터에 붙여준다.
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T
league_df = pd.concat([league_df, league_entries_df], axis=1) 
league_df

,index,tier,leagueId,queue,name,entries,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,0,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'KEKtyDMgTJUcPhfsHztLtpU98KyIeC...,KEKtyDMgTJUcPhfsHztLtpU98KyIeC1G5NYnHix12F0lajQ,윤용호,1028,I,287,219,False,False,False,True
1,1,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': '97XkEPTUfhI8BK192Bakb9Xeqhd9BX...,97XkEPTUfhI8BK192Bakb9Xeqhd9BXP5LpHrYnHipvtcfB...,앙 맛있엉,1150,I,463,390,True,False,False,False
2,2,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': '3OHO1cpkydLO2EdSKmigx_vy79Tg_k...,3OHO1cpkydLO2EdSKmigx_vy79Tg_kuuIVrwrqBvv5kWAhwL,T1 Smash1,1135,I,529,454,True,False,False,False
3,3,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'EXazcoG7DZN9OJL05TN0cXFv9f0QoM...,EXazcoG7DZN9OJL05TN0cXFv9f0QoMhEuMTU6yepWe1w4FY,Washing hands,951,I,505,455,False,False,True,True
4,4,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'jxQK0sGhLCW23SaTJ7m4d2EV23-Rkr...,jxQK0sGhLCW23SaTJ7m4d2EV23-Rkrk2rgj03SIqcseGPEad,sryteam01,997,I,819,742,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'qHT9-KIZ5QG4gvuD20fjMwqDLV2Goz...,qHT9-KIZ5QG4gvuD20fjMwqDLV2Goz2gOnGatc-IAjCpMlg,Radiohead,1291,I,1047,964,True,False,False,False
296,296,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'COgQT7YJapv-WkU7gUg8t8MGYdWxh5...,COgQT7YJapv-WkU7gUg8t8MGYdWxh5cqvQWYfGqdCFabeX...,Cry Outt,1019,I,224,167,True,False,False,True
297,297,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'DjeJeRD0dipXpP7yHeWtshisALDict...,DjeJeRD0dipXpP7yHeWtshisALDictgoWBtXaj4Y3_i6h6k,전설의롤1234,951,I,363,307,True,False,False,False
298,298,CHALLENGER,05fb99f4-e149-3133-a78e-821597582f9d,RANKED_SOLO_5x5,Rengar's Swashbucklers,{'summonerId': 'sBjz6K6MuPtOo2P7DMzhwMFJtQ5xhS...,sBjz6K6MuPtOo2P7DMzhwMFJtQ5xhSz7APlN0InwR91WSS...,Academy T1,1104,I,389,305,True,False,False,True


In [ ]:
#하지만 위의 데이터는 가독성이 매우 떨어진다.
#따라서 효율적인 데이터 관리를 위해, 필요없는 데이터 column은 drop을 이용하여 없애준다.
league_df_mod = league_df.drop(['index','leagueId','queue','name','entries','rank','inactive','freshBlood','hotStreak'], axis=1)
league_df_mod
#아래와 같이 'summonerName', 앞서 (1)에서 진행한 방법을 이용하기 위한 정보를 얻었음을 알 수 있다.

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran
0,CHALLENGER,KEKtyDMgTJUcPhfsHztLtpU98KyIeC1G5NYnHix12F0lajQ,윤용호,1028,287,219,False
1,CHALLENGER,97XkEPTUfhI8BK192Bakb9Xeqhd9BXP5LpHrYnHipvtcfB...,앙 맛있엉,1150,463,390,True
2,CHALLENGER,3OHO1cpkydLO2EdSKmigx_vy79Tg_kuuIVrwrqBvv5kWAhwL,T1 Smash1,1135,529,454,True
3,CHALLENGER,EXazcoG7DZN9OJL05TN0cXFv9f0QoMhEuMTU6yepWe1w4FY,Washing hands,951,505,455,False
4,CHALLENGER,jxQK0sGhLCW23SaTJ7m4d2EV23-Rkrk2rgj03SIqcseGPEad,sryteam01,997,819,742,False
...,...,...,...,...,...,...,...
295,CHALLENGER,qHT9-KIZ5QG4gvuD20fjMwqDLV2Goz2gOnGatc-IAjCpMlg,Radiohead,1291,1047,964,True
296,CHALLENGER,COgQT7YJapv-WkU7gUg8t8MGYdWxh5cqvQWYfGqdCFabeX...,Cry Outt,1019,224,167,True
297,CHALLENGER,DjeJeRD0dipXpP7yHeWtshisALDictgoWBtXaj4Y3_i6h6k,전설의롤1234,951,363,307,True
298,CHALLENGER,sBjz6K6MuPtOo2P7DMzhwMFJtQ5xhSz7APlN0InwR91WSS...,Academy T1,1104,389,305,True


In [ ]:
#api는 주기적으로 갱신해줘야하기 때문에 저장해준다.
league_df.to_csv('챌린저데이터.csv',index=False)
league_df_mod.to_csv('챌린저데이터정제.csv',index=False)

In [ ]:
#앞서 실행한 (1)의 방법을 이용해, for문을 이용하여 300명의 플레이어에 대한 puuid를 모두 가져온다.
#이 때 'veteran' column 역시 필요 없기 때문에, 이 곳에 덮어쓴 후 column의 이름을 바꿔준다.
import time
for i in range(len(league_df_mod)):
    try:
        sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(sohwan)
        
        while r.status_code == 429: #응답이 가능할 때
            time.sleep(5)
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)
    
        puuid = r.json()['puuid']
        league_df.iloc[i, -1] = puuid
        league_df_mod.iloc[i,-1] = puuid
    
    except:
        pass

In [ ]:
#역시 저장해준다.
league_df_rename = league_df_mod.rename(columns={'veteran':'puuid'})
league_df_rename.to_csv('챌린저데이터puuid.csv',index=False)

In [ ]:
#전적검색을 위해 필요한 것은 puuid뿐이기 때문에, 이제 이 것만 사용하기 위해 변수를 지정해준다.
IDsearch = league_df_rename['puuid']

In [ ]:
#전적을 검색해주는 matchid api를 이용하여 각 모든 puuid에 대해 50개의 경기의 matchid를 가져온다.
#여기서 matchid란 각 경기에 대한 고유 id이다.
#따라서 50*300 = 15000개의 경기라고 예상되지만,
#서로 겹치는 플레이를 한 게임이 존재하기 때문에, 특히나 최상위권은 대부분의 경우 겹치기 때문에,
#중복되는 경기를 없애준다.
import time

matchid = []
cnt = 0
for i in range(len(IDsearch)):
    try:
        time.sleep(5)
        match0 = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + league_df_rename['puuid'].iloc[i]  + '/ids?count=50' + '&api_key=' + api_key
        r = requests.get(match0)
        r = json.loads(r.content)
        matchid.extend(r)
        cnt += 1
        print('complete '+cnt+' times')
    except:
        pass
matchid = list(set(matchid))

In [ ]:
len(matchid)
#예상했던 대로 15000의 약 2/3인 9734개의 데이터를 얻었다.

9734

In [ ]:
import pickle
with open("챌린저matchid.pkl","wb") as f:
    pickle.dump(matchid, f)
#저장해준다.

In [ ]:
# 다시 불러오기
# with open("챌린저matchid.pkl","rb") as f:
#     matchid = pickle.load(f)

In [ ]:
#matchid의 예시이다. 앞에 한국서버라고 명시되어있음을 확인할 수 있다.
matchid[0]

'KR_6249775015'

In [ ]:
#이제 matchid를 이용하여 match의 세부 정보를 가져와주는 api를 이용한다.
#973개의 데이터 기준 약 3시간이 소요되는 작업이기 때문에,
#전체 데이터를 불러오지 못하고 얻어낸 데이터의 1/10만 사용하였다.
import time

game_data = pd.DataFrame()
for i in range(len(matchid)//10):
    time.sleep(5)
    api_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid[i] + '?api_key=' + api_key
    r = requests.get(api_url)
    r = json.loads(r.content)
    r = json_normalize(r)
    game_data = pd.concat([game_data,r])
game_data

,metadata.dataVersion,metadata.matchId,metadata.participants,info.gameCreation,info.gameDuration,info.gameEndTimestamp,info.gameId,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.participants,info.platformId,info.queueId,info.teams,info.tournamentCode
0,2,KR_6249775015,[MDsNVNJQ7KEcIUuZAIppCPptcol5cBBbiYUXicu9iGLSu...,1670509251829,965,1670510270118,6249775015,CLASSIC,teambuilder-match-6249775015,1670509304739,MATCHED_GAME,12.23.483.5208,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,420,"[{'bans': [{'championId': 236, 'pickTurn': 1},...",
0,2,KR_6232771583,[AHRFVu27Hpp1qEo0-zlMcjjLUBa8kZJ-lPB8gk_xt4M07...,1669391086872,932,1669392061913,6232771583,CLASSIC,teambuilder-match-6232771583,1669391129164,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,420,"[{'bans': [{'championId': -1, 'pickTurn': 1}, ...",
0,2,KR_6241463014,[l0CopLyefQaDP1lqpwgwbodsYGiA_5tXPLF_cDviH6heq...,1669979542861,1284,1669980844073,6241463014,CLASSIC,teambuilder-match-6241463014,1669979559297,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 2, 'assist...",KR,420,"[{'bans': [{'championId': 555, 'pickTurn': 1},...",
0,2,KR_6199028990,[KoEIXozbbO2Wsr08UYvJ-4M-rNj_AjajZGHBRfqJkveTx...,1667401773089,883,1667402692609,6199028990,ARAM,teambuilder-match-6199028990,1667401808715,MATCHED_GAME,12.21.477.420,12,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,450,"[{'bans': [], 'objectives': {'baron': {'first'...",
0,2,KR_6238507870,[Tb-YCfbqj6weLp5AViv3NMs0mh1cyM6UviDPJTS39tULM...,1669746075956,2050,1669748150485,6238507870,CLASSIC,teambuilder-match-6238507870,1669746099492,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 3, 'assist...",KR,430,"[{'bans': [], 'objectives': {'baron': {'first'...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2,KR_6252785617,[SHrqS3pSrVc2Sl5VPAYLxRpF5Tg18nRABH_Nur25pQe2J...,1670671455394,701,1670672231334,6252785617,ARAM,teambuilder-match-6252785617,1670671529719,MATCHED_GAME,12.23.483.5208,12,"[{'allInPings': 0, 'assistMePings': 1, 'assist...",KR,720,"[{'bans': [], 'objectives': {'baron': {'first'...",
0,2,KR_6197641493,[jKdOVqOVLVfklp77ztIaEekxurdi5Fbym8HHIMTUCnn6C...,1667307394491,1462,1667308937638,6197641493,CLASSIC,teambuilder-match-6197641493,1667307474671,MATCHED_GAME,12.20.474.8882,11,"[{'assists': 4, 'baronKills': 0, 'basicPings':...",KR,420,"[{'bans': [{'championId': 104, 'pickTurn': 1},...",
0,2,KR_6259751709,[k8ancec0ffUK8pM8h8PF01yuCDerjc_6DwAlU4pkPcTg9...,1671039209038,932,1671040156417,6259751709,CLASSIC,teambuilder-match-6259751709,1671039223613,MATCHED_GAME,12.23.483.5208,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,420,"[{'bans': [{'championId': 236, 'pickTurn': 1},...",
0,2,KR_6250730329,[YjVHpxezxkVUiMGzzu3rA4ibIiIXGzXKp0mfdyq03z3wT...,1670579414424,918,1670580355483,6250730329,CLASSIC,teambuilder-match-6250730329,1670579437347,MATCHED_GAME,12.23.483.5208,11,"[{'allInPings': 0, 'assistMePings': 4, 'assist...",KR,420,"[{'bans': [{'championId': 236, 'pickTurn': 1},...",


In [ ]:
#역시 경기 정보에서 우리가 필요한 것은 승률에 연관되어있는 것만이기 때문에, 해당 column만 추출해준다.
#여기서 gameDuration은 경기 시간, participants는 플레이어정보, teams는 팀 전체 정보이다.
game_mod = game_data[['info.gameDuration','info.participants','info.teams']]

In [ ]:
#롤은 5vs5 팀게임이기 때문에,
#개인의 정보보다는, 팀 전체의 정보와 게임 시간이 더 많은 영향을 미친다고 알려져있다.
#따라서 이 프로젝트에서는 경기 시간과 팀 정보만을 학습에 이용하기로 한다.
teams = list(game_mod['info.teams'])
times = list(game_mod['info.gameDuration'])
data_time = pd.DataFrame(times,columns=['gameDuration'])

In [ ]:
#레드팀에 대한 정보이다.
team1_df = pd.DataFrame()
col = []
teams[0][0].pop('bans',None) #필요없는 정보는 없애준다.
target = list(teams[0][0].values())[0]
for k, v in target.items():
    col.append(k+list(v.keys())[0])
    col.append(k+list(v.keys())[1])
col.append('teamId')
col.append('win')

for i in range(len(teams)):
    entry = []
    teams[i][0].pop('bans',None)
    target = list(teams[i][0].values())[0]
    for k, v in target.items():
        entry.append(list(v.values())[0])
        entry.append(list(v.values())[1])
    entry.append(teams[i][0]['teamId'])
    entry.append(teams[i][0]['win'])
    team1 = pd.DataFrame(entry, index=col).T
    team1_df = pd.concat([team1_df, team1])
team1_df.index = range(len(team1_df))

#블루팀에 대한 정보이다.
team2_df = pd.DataFrame()
for i in range(len(teams)):
    entry = []
    teams[i][1].pop('bans',None)
    target = list(teams[i][1].values())[0]
    for k, v in target.items():
        entry.append(list(v.values())[0])
        entry.append(list(v.values())[1])
    entry.append(teams[i][1]['teamId'])
    entry.append(teams[i][1]['win'])
    team2 = pd.DataFrame(entry, index=col).T
    team2_df = pd.concat([team2_df, team2])
team2_df.index = range(len(team2_df))

#하지만, 게임의 특성 상 이긴 팀만 분석하면 자동으로 진 팀도 분석이 되기 때문에(symmetric)
#한 쪽 팀의 데이터만 이용하겠다.

data_team = pd.concat([team1_df, data_time],axis=1)
data_team
#분석을 위한 데이터 준비가 끝났다.

,baronfirst,baronkills,championfirst,championkills,dragonfirst,dragonkills,inhibitorfirst,inhibitorkills,riftHeraldfirst,riftHeraldkills,towerfirst,towerkills,teamId,win,gameDuration
0,False,0,True,29,False,0,False,0,False,0,True,3,100,True,965
1,False,0,False,13,False,0,False,0,False,0,False,0,100,False,932
2,True,1,True,26,True,2,False,0,False,1,False,5,100,True,1284
3,False,0,True,32,False,0,False,0,False,0,False,2,100,False,883
4,True,2,True,39,False,4,True,2,False,0,True,10,100,True,2050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,False,0,True,31,False,0,True,1,False,0,True,4,100,True,701
969,False,0,False,12,True,2,False,0,True,1,True,3,100,False,1462
970,False,0,True,22,True,1,False,0,True,1,True,3,100,True,932
971,False,0,True,11,False,0,False,0,False,0,False,0,100,False,918


In [ ]:
data_team.to_csv('챌린저데이터최종.csv',index=False)
#매우 오래걸리므로 미리 저장해둔다.

### 3. sklearn을 이용한 머신러닝

#### 3-1. 데이터 정제 과정

In [ ]:
#이제 데이터를 구분하자. 우리는 학습할 '팀적 데이터 및 경기 시간'을 기반으로 '승률'을 예측할 것이다.
#학습할 데이터와, 결과를 예측할 데이터를 구분한다.
Input = list(data.columns)[:-3]
Input.append('gameDuration')

data_Input = data_team[Input]
data_Target = data_team['win']

In [ ]:
#Bool타입의 데이터들은 라벨링을 이용하여 0과 1로 바꿔준다.
from sklearn.preprocessing import LabelEncoder
for i in range(0,13):
    le = LabelEncoder()
    y = list(data_Input.iloc[:,i])
    
    le.fit(y)
    y2 = le.transform(y) 
    
    data_Input.iloc[:,i] = y2

C:\Users\hongg\AppData\Local\Temp\ipykernel_5560\467881749.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_Input.iloc[:,i] = y2


In [ ]:
#결과값 역시 bool이기 때문에 바꿔준다.
bool_2_int = {True : 0, False: 1}
data_Target = data_Target.map(bool_2_int).tolist()

In [ ]:
#Input과 Target을 각각 train용도, 그리고 test용도로 3:1의 비율로 랜덤하게 섞어서 나눠준다.
from sklearn.model_selection import train_test_split
import numpy as np

Input_train, Input_test, Target_train, Target_test = train_test_split(data_Input, np.array(data_Target), test_size=0.25, stratify=np.array(data_Target), random_state=123456)

#### 3-2. 랜덤포레스트를 이용한 머신러닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RF

rf = RF(n_estimators=100, oob_score=True, random_state=123456)
rf.fit(Input_train, Target_train)

from sklearn.metrics import accuracy_score

predicted_rf = rf.predict(Input_test)
accuracy = accuracy_score(Target_test, predicted_rf)
print(f'평균 승률 예측 정확도: {accuracy:.3}%')

평균 승률 예측 정확도: 0.922%


#### 3-3. 그래디언트 부스팅을 이용한 머신러닝

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier as GBC
clf_gbc = GBC()
clf_gbc.fit(Input_train, Target_train)

predicted_gbc = clf_gbc.predict(Input_test)
accuracy = accuracy_score(Target_test,predicted_gbc)
print(f'평균 승률 예측 정확도: {accuracy:.3}%')

평균 승률 예측 정확도: 0.906%


In [ ]:
#매우 높은 승률 예측 정확도를 보여주기 때문에, 신뢰할 수 있는 프로그램을 만들었다고 결론지을 수 있다.
#(한계점)2개의 방법이 각각 어떤 요소를 더 중점적으로 생각했기 때문에 정확도에 차이가 났는지를
#알 수 있다면 더욱 좋은 프로그램이 되었을 것 같다.

#또한 높은 정확도가 말해주는 것은,
#앞서 말한대로 경기 시간과, 팀적인 요소들이 승률에 밀접한 연관이 있음을 알 수 있다.

### 4. 사용자 ID에 직접 적용

In [ ]:
#이제 직접 사용자들의 경기 결과를 가져와 검색해보자.

#본인의 플레이 데이터셋을 가져와, 승패 여부를 잘 예측했는지 알아보자.

In [ ]:
MYID = '호걸쾌남' # <-- 이 곳에 본인의 아이디를 입력하면 된다.

In [ ]:
# puuid 얻기
api_key = 'RGAPI-5ae606a0-3159-4daf-88a4-8e6ef6226c39'
sohwan = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + MYID +'?api_key=' + api_key
r = requests.get(sohwan)
player = json.loads(r.content)
userpuuid = player['puuid']

In [ ]:
# puuid로부터 -> matchid 얻기
user_matchid = []
match = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + userpuuid  + '/ids?count=20' + '&api_key=' + api_key
r = requests.get(match)
r = json.loads(r.content)
user_matchid.extend(r)
user_matchid = list(set(user_matchid))

In [ ]:
# matchid로부터 --> 게임 데이터 얻기
user_game_data = pd.DataFrame()
for i in range(len(user_matchid)):
    time.sleep(5)
    api_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + user_matchid[i] + '?api_key=' + api_key
    r = requests.get(api_url)
    r = json.loads(r.content)
    r = json_normalize(r)
    user_game_data = pd.concat([game_data,r])
user_game_data

,metadata.dataVersion,metadata.matchId,metadata.participants,info.gameCreation,info.gameDuration,info.gameEndTimestamp,info.gameId,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.participants,info.platformId,info.queueId,info.teams,info.tournamentCode
0,2,KR_6215950268,[KJroq74_0JEJqBgEplOmcVpc_sk0ZbXWrdFBLyc2rDvXH...,1668355938101,1688,1668357681311,6215950268,CLASSIC,teambuilder-match-6215950268,1668355993201,MATCHED_GAME,12.21.477.420,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 360, 'pickTurn': 1},...",
0,2,KR_6243165837,[ClmMR64Gw6y8rJq1tGmavTjYug1eEFa3jZtnxrUZaQoQn...,1670074311713,1287,1670075622745,6243165837,CLASSIC,teambuilder-match-6243165837,1670074335413,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 1, 'assist...",KR,440,"[{'bans': [{'championId': 35, 'pickTurn': 1}, ...",
0,2,KR_6245035425,[lDVA2KrVH9gTzi8kxo7Emxr8TdPLsMIcqxidD90dTTc9M...,1670167255029,2077,1670169359003,6245035425,CLASSIC,teambuilder-match-6245035425,1670167281046,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 238, 'pickTurn': 1},...",
0,2,KR_6218457456,[_PPBMAFnjXUZ_hES5UfrpnWEZH8503ohaXojJvYAC5rlS...,1668518389034,2099,1668520512495,6218457456,CLASSIC,teambuilder-match-6218457456,1668518412540,MATCHED_GAME,12.21.477.420,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 53, 'pickTurn': 1}, ...",
0,2,KR_6217834744,[MVTk_z7dWHYe4SerKZZUS7uH7AcOny0RGuT4b8B_IP05v...,1668491018071,1118,1668492161419,6217834744,URF,teambuilder-match-6217834744,1668491043334,MATCHED_GAME,12.21.477.420,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,1900,"[{'bans': [{'championId': 81, 'pickTurn': 1}, ...",
0,2,KR_6215986335,[-1Y4off0YTCh44mI5nTjkNVYw0xxLn3OO0ZGmQLWw3SlM...,1668357934963,1318,1668359272365,6215986335,CLASSIC,teambuilder-match-6215986335,1668357954126,MATCHED_GAME,12.21.477.420,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 238, 'pickTurn': 1},...",
0,2,KR_6243229530,[TnFjhEZh5cvHeDhMdrfL52n-xsDiFiPUNrU5kAdBY3fkW...,1670075907610,1601,1670077546775,6243229530,CLASSIC,teambuilder-match-6243229530,1670075945491,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 238, 'pickTurn': 1},...",
0,2,KR_6244969560,[qETkQ4919kwGlVtkIzOdsr2fAkSZBO2SVGUxGn3h65sdD...,1670164674484,2261,1670166959785,6244969560,CLASSIC,teambuilder-match-6244969560,1670164698513,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 157, 'pickTurn': 1},...",
0,2,KR_6218586194,[Y09phh0-cfBJseh7QFJDzDKPWFFYn_MiQX5z2EZKe0JDh...,1668522249032,996,1668523270263,6218586194,CLASSIC,teambuilder-match-6218586194,1668522274267,MATCHED_GAME,12.21.477.420,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 360, 'pickTurn': 1},...",
0,2,KR_6244415010,[_PPBMAFnjXUZ_hES5UfrpnWEZH8503ohaXojJvYAC5rlS...,1670147188093,2013,1670149232702,6244415010,CLASSIC,teambuilder-match-6244415010,1670147219253,MATCHED_GAME,12.22.479.5277,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,440,"[{'bans': [{'championId': 35, 'pickTurn': 1}, ...",


In [ ]:
user_game_mod = user_game_data[['info.gameDuration','info.participants','info.teams']]
user_teams = list(user_game_mod['info.teams'])
user_times = list(user_game_mod['info.gameDuration'])
user_data_time = pd.DataFrame(user_times,columns=['gameDuration'])

In [ ]:
user_team_df = pd.DataFrame()
col = []
user_teams[0][0].pop('bans',None)
target = list(user_teams[0][0].values())[0]
for k, v in target.items():
    col.append(k+list(v.keys())[0])
    col.append(k+list(v.keys())[1])
col.append('teamId')
col.append('win')

for i in range(len(user_teams)):
    entry = []
    user_teams[i][0].pop('bans',None)
    target = list(user_teams[i][0].values())[0]
    for k, v in target.items():
        entry.append(list(v.values())[0])
        entry.append(list(v.values())[1])
    entry.append(user_teams[i][0]['teamId'])
    entry.append(user_teams[i][0]['win'])
    user_team = pd.DataFrame(entry, index=col).T
    user_team_df = pd.concat([user_team_df, user_team])

user_team_df.index = range(len(user_team_df))
user_team_df = pd.concat([user_team_df, user_data_time],axis=1)
user_team_df

,baronfirst,baronkills,championfirst,championkills,dragonfirst,dragonkills,inhibitorfirst,inhibitorkills,riftHeraldfirst,riftHeraldkills,towerfirst,towerkills,teamId,win,gameDuration
0,True,1,False,51,False,2,True,2,True,2,True,9,100,True,1688
1,False,0,True,9,False,0,False,0,False,0,False,0,100,False,1287
2,False,0,True,54,True,4,True,2,True,1,True,11,100,True,2077
3,False,0,True,21,True,1,False,0,False,0,False,2,100,False,2099
4,False,0,True,43,True,2,False,0,False,0,True,2,100,False,1118
5,False,0,False,12,False,0,False,0,False,0,False,1,100,False,1318
6,True,1,False,41,True,3,True,1,True,1,True,7,100,True,1601
7,False,0,False,41,True,1,False,0,False,0,False,4,100,False,2261
8,False,0,True,28,True,1,False,0,True,1,True,2,100,True,996
9,True,1,False,44,False,3,False,1,False,1,False,7,100,True,2013


In [ ]:
#데이터 정제
Input = list(data.columns)[:-3]
Input.append('gameDuration')
data_Input = user_team_df[Input]
data_Target = user_team_df['win']
from sklearn.preprocessing import LabelEncoder
for i in range(0,13):
    le = LabelEncoder()
    y = list(data_Input.iloc[:,i])
    
    le.fit(y)
    y2 = le.transform(y) 
    
    data_Input.iloc[:,i] = y2
bool_2_int = {True : 0, False: 1}
data_Target = data_Target.map(bool_2_int).tolist()

C:\Users\hongg\AppData\Local\Temp\ipykernel_5560\3756501114.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_Input.iloc[:,i] = y2


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

user_Input_train, user_Input_test, user_Target_train, user_Target_test = train_test_split(data_Input, np.array(data_Target), test_size=0.9, stratify=np.array(data_Target), random_state=123456)

In [ ]:
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score

rf = RF(n_estimators=100, oob_score=True, random_state=123456)
rf.fit(Input_train, Target_train) #기존의 데이터로 학습
user_predicted_rf = rf.predict(user_Input_test)
accuracy = accuracy_score(user_Target_test, user_predicted_rf)
print(f'챌린저 게임을 통해 학습한 승패예측과 비교한 정확도는: {accuracy:.3}%')

챌린저 게임을 통해 학습한 승패예측과 비교한 정확도는: 0.778%


In [ ]:
#그래디언트 부스팅
from sklearn.ensemble import GradientBoostingClassifier as GBC
clf_gbc = GBC()
clf_gbc.fit(Input_train, Target_train) #기존의 데이터로 학습

user_predicted_gbc = clf_gbc.predict(user_Input_test)
accuracy = accuracy_score(user_Target_test,user_predicted_gbc)
print(f'챌린저 게임을 통해 학습한 승패예측과 비교한 정확도는: {accuracy:.3}%')

챌린저 게임을 통해 학습한 승패예측과 비교한 정확도는: 0.611%


In [ ]:
#티어(게임 수준)의 차이로 인해, 최상위권에서 적용되는 통계적인 정보는
#하위 티어로 내려올수록 적중률이 낮아지고, 이는 게임 수준의 차이때문에 당연하다.
#실력이 올라갔는지 정확한 수치로 알아보고 싶을 때, 더 많은 플레이 이후에
#새로운 게임 데이터 정보를 프로그램에 돌려보고,
#이 때의 결과가 더 높게 나온다면 실력이 올랐다고 판단할 수 있을 것이다.

### 결론

앞선 챌린저 게임의 정확도는 랜덤포레스트 방법이 미세하게 더 높았다.  
이는 랜덤포레스트 방법을 통해 학습할 때 높은 비중을 차지한 요소가 승률에 미세하게 더 큰 영향을 미친다-  
라고 결론낼 수 있다.  
  
  
그리고 또한, 이를 하위 티어인 본인의 아이디의 경기를 이용하여 테스트 해보았을 때,  
정확도 역시 랜덤포레스트가 훨씬 높은 정확도를 보여주고 있었다. 이는 챌린저 경기에 비해 유의미한 차이이다.  
하지만 그렇다고해서 그래디언트 부스팅의 방법이 좋지 않다고 판단할 수는 없다.  
왜냐하면, 랜덤포레스트를 통한다면 - 하위티어는 경기 양상이 규칙적이지 않고 무작위임에도 불구하고-  
승률을 꽤나 높은 정확도로 예측할 수 있다. - 라는 결론을 얻을 수 있는 반면,  
  
그래디언트 부스팅의 방법을 통한다면 - 챌린저 경기와 비교했을 때 차이가 나는 부분이 많았기 때문에 -  
승률을 잘 예측하지 못한 것이다 - 라는 결론을 얻을 수 있다.  

1- 랜덤포레스트를 이용하면 승률 예측이라는 목적 자체에 부합한 결과를 얻을 수 있다.  
2- 그래디언트 부스팅을 이용하면 챌린저 경기와 나의 경기가 무엇이 다른지를 알 수 있다.  
   즉, 경기를 어떻게 바꿔야 더 실력이 올라갈 수 있는지 알 수 있다.  
   
(한계점) 따라서, 각 방법이 어떤 요소를 가중치로 두었는지를 알 수 있다면, 실력을 높이기 위한 요소의 범위를 더욱 좁힐 수 있을 것이다.